# Welcome to team Klima Kämpfer's notebook for the WiDS datathon 2022

## What data are we exploring here? 

### The data in this notebook looks at energy efficiency of buildings in the United States. The goal is to find the best predictors of energy efficienct, that is those predictors that are most correlated with the outcome variable and give us the strongest model in terms of predicting the outcome variable. The dataset can be downloaded [here](https://www.kaggle.com/c/widsdatathon2022/data) 

## 1. Data cleaning and preparation

In [178]:
import pandas as pd
import numpy as np 

In [179]:
#loading the data
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

### Cleaning train set (steps need to be repeated for test set)

In [180]:
#check how many rows and colums we have
print('Train set rows:',train.shape[0])
print('Train set columns:',train.shape[1])

Train set rows: 75757
Train set columns: 64


### Object/string type variables need to be encoded to integer types

In [181]:
from sklearn.preprocessing import LabelEncoder

# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in orig column
train['building_class'] = labelencoder.fit_transform(train['building_class'])
train['facility_type'] = labelencoder.fit_transform(train['facility_type'])
train['State_Factor'] = labelencoder.fit_transform(train['State_Factor'])

test['building_class'] = labelencoder.fit_transform(test['building_class'])
test['facility_type'] = labelencoder.fit_transform(test['facility_type'])
test['State_Factor'] = labelencoder.fit_transform(test['State_Factor'])

In [182]:
# Use simple imputer for numerical columns
from sklearn.impute import SimpleImputer
null_cols=['year_built', 'energy_star_rating', 'direction_max_wind_speed', 'direction_peak_wind_speed', 'max_wind_speed',
          'days_with_fog']
imp = SimpleImputer(missing_values=np.nan, strategy='median')

train[null_cols]  = imp.fit_transform(train[null_cols])
test[null_cols]  = imp.transform(test[null_cols])
test = test.fillna(0)

In [183]:
#detect outliers, outliers are one of the primary reasons for resulting in a less accurate model
#it's a good idea to remove them

Q1 = train.iloc[0:49048,0:27].quantile(0.25) # first quartile (25%)
Q3 = train.iloc[0:49048,0:27].quantile(0.75) # third quartile (75%)
IQR = Q3 - Q1 # Interquartile range (IQR)
print(IQR)

Year_Factor                2.000000
State_Factor               1.000000
building_class             1.000000
facility_type              5.000000
floor_area            117161.500000
year_built                45.000000
energy_star_rating        11.000000
ELEVATION                 33.600000
january_min_temp           9.000000
january_avg_temp           8.725806
january_max_temp           6.000000
february_min_temp         17.000000
february_avg_temp         10.620690
february_max_temp          9.000000
march_min_temp            15.000000
march_avg_temp            12.983871
march_max_temp            16.000000
april_min_temp             7.000000
april_avg_temp             2.433333
april_max_temp             8.000000
may_min_temp               7.000000
may_avg_temp               2.306452
may_max_temp               3.000000
june_min_temp              5.000000
june_avg_temp              1.666667
june_max_temp              4.000000
july_min_temp              3.000000
dtype: float64


In [184]:
# Checking Train set size before removing outliers
print('Train set rows:',train.shape[0])
print('Train set columns:',train.shape[1])

Train set rows: 75757
Train set columns: 64


In [185]:
# train = train[~((train < (Q1 - 1.5 * IQR)) |(train > (Q3 + 1.5 * IQR))).any(axis=1)]

In [186]:
# Checking Train set size after removing outliers
print('Train set rows:',train.shape[0])
print('Train set columns:',train.shape[1])

Train set rows: 75757
Train set columns: 64


# Cleaning testing set

### Test Set: Object/string type variables need to be encoded to integer types

In [187]:
#test features
X = train[['energy_star_rating', 'january_min_temp', 'january_avg_temp', 'february_avg_temp', 'february_min_temp', 
'march_avg_temp', 'march_min_temp', 'april_avg_temp','april_min_temp', 'february_max_temp', 'november_min_temp', 
'july_min_temp','september_avg_temp', 'snowdepth_inches', 'precipitation_inches','august_avg_temp', 'days_below_20F',
'june_avg_temp','cooling_degree_days','july_avg_temp','days_below_30F','heating_degree_days','snowfall_inches',
'facility_type','floor_area','year_built', 'State_Factor', 'days_with_fog', 'direction_max_wind_speed']] 
y = train['site_eui']

In [188]:
#train features

test_features = test[['energy_star_rating', 'january_min_temp', 'january_avg_temp', 'february_avg_temp', 'february_min_temp', 
'march_avg_temp', 'march_min_temp', 'april_avg_temp','april_min_temp', 'february_max_temp', 'november_min_temp', 
'july_min_temp','september_avg_temp', 'snowdepth_inches', 'precipitation_inches','august_avg_temp', 'days_below_20F',
'june_avg_temp','cooling_degree_days','july_avg_temp','days_below_30F','heating_degree_days','snowfall_inches','facility_type',
'floor_area','year_built', 'State_Factor','days_with_fog','direction_max_wind_speed']]

# TF Deep NN Build

In [189]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.8.0


In [190]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X)

In [191]:
model = keras.Sequential([
      normalizer,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='elu'),
      layers.Dense(1)
  ])

model.compile(loss='mean_squared_error',
                optimizer=tf.keras.optimizers.Adam(0.001), 
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [192]:
model.fit(X, y, validation_split=0.3, epochs=30, verbose=1)

Epoch 1/30
1658/1658 [==============================] - 2s 1ms/step - loss: 2883.6458 - root_mean_squared_error: 53.6996 - val_loss: 4573.6777 - val_root_mean_squared_error: 67.6290
Epoch 2/30
1658/1658 [==============================] - 2s 1ms/step - loss: 2588.5718 - root_mean_squared_error: 50.8780 - val_loss: 4029.4692 - val_root_mean_squared_error: 63.4781
Epoch 3/30
1658/1658 [==============================] - 2s 1ms/step - loss: 2558.0198 - root_mean_squared_error: 50.5769 - val_loss: 4330.8594 - val_root_mean_squared_error: 65.8093
Epoch 4/30
1658/1658 [==============================] - 2s 1ms/step - loss: 2526.1838 - root_mean_squared_error: 50.2612 - val_loss: 4665.7207 - val_root_mean_squared_error: 68.3061
Epoch 5/30
1658/1658 [==============================] - 2s 1ms/step - loss: 2501.1782 - root_mean_squared_error: 50.0118 - val_loss: 4655.3877 - val_root_mean_squared_error: 68.2304
Epoch 6/30
1658/1658 [==============================] - 2s 1ms/step - loss: 2480.2441 - ro

# Setting up model for test submission

In [134]:
#Checking size of the test set
print('Test set rows:',test.shape[0])
print('Test set columns:',test.shape[1])

Test set rows: 9705
Test set columns: 63


In [193]:
y_hat_official = model.predict(test_features) # Getting predicted values for submission
y_hat_official = y_hat_official.flatten() # Flattening the output tensor to a numpy array

In [194]:
submission_predicted = pd.DataFrame({'site_eui': y_hat_official}) # Creating a dataframe of predicted values
id_predicted = pd.DataFrame({'id': test['id']}) # Creating a dataframe of test ids

In [195]:
submission_file_df = pd.concat([id_predicted, submission_predicted], axis=1) # Combining the two dataframes

In [196]:
print("Submission file shape:", submission_file_df.shape)
submission_file_df.head()

Submission file shape: (9705, 2)


,id,site_eui
0,75757,235.962006
1,75758,198.178589
2,75759,278.712067
3,75760,245.982117
4,75761,238.155182


In [197]:
submission_file_df.to_csv('submission_file_tf_v9.csv', index=False) # Saving the submission file